In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
#print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.
import pandas as pd
import numpy as np
from helpers import calibrate_train_clfs, load_data, get_baseline, print_results, run_clf

Using TensorFlow backend.


In [25]:
keep_columns =  ['IsTraining','FTR', 'Div', 'Date', 'HomeTeam', 'AwayTeam', 
    'away_away_team_corners_against', 'away_away_team_corners_for',	'away_away_team_goals_against',	'away_away_team_goals_for',	
    'away_away_team_possession',	'away_away_team_shotsoff_against',	'away_away_team_shotsoff_for',	'away_away_team_shotson_against',
    	'away_away_team_shotson_for',	'away_direct_team_corners_against',	'away_direct_team_corners_for',	'away_direct_team_goals_against',
        'away_direct_team_goals_for',	'away_direct_team_possession',	'away_direct_team_shotsoff_against',	'away_direct_team_shotsoff_for',
        'away_direct_team_shotson_against',	'away_direct_team_shotson_for',	'home_direct_team_corners_against',	'home_direct_team_corners_for',	
        'home_direct_team_goals_against',	'home_direct_team_goals_for',	'home_direct_team_possession',	'home_direct_team_shotsoff_against',
        'home_direct_team_shotsoff_for',	'home_direct_team_shotson_against',	'home_direct_team_shotson_for',	
        'home_home_team_corners_against',	'home_home_team_corners_for',	'home_home_team_goals_against',	'home_home_team_goals_for',	
        'home_home_team_possession',	'home_home_team_shotsoff_against',	'home_home_team_shotsoff_for',	'home_home_team_shotson_against',
                 'home_home_team_shotson_for','Referee' 
]
#'FTHG'

drop_columns = ['B365H', 'B365D', 'B365A',
    'AC', 'AF', 'AR', 'AS', 'AST', 'AY', 'BWA', 'BWD', 'BWH', 'Bb1X2', 'BbAH', 'BbAHh', 'BbAv<2.5', 'BbAv>2.5', 'BbAvA', 'BbAvAHA', 
    'BbAvAHH', 'BbAvD', 'BbAvH', 'BbMx<2.5', 'BbMx>2.5', 'BbMxA', 'BbMxAHA', 'BbMxAHH', 'BbMxD', 'BbMxH', 'BbOU', 'FTAG', 'HC', 'HF', 'HR', 'HS', 'HST', 
    'HTAG', 'HTHG', 'HTR', 'HY', 'IWA', 'IWD', 'IWH', 'LBA', 'LBD', 'LBH', 'PSA', 'PSCA', 'PSCD', 'PSCH', 'PSD', 'PSH', 
    'VCA', 'VCD', 'VCH', 'WHA', 'WHD', 'WHH', 'GoalFirstHalf', 'GoalSecondHalf', 'SHHG', 'SHAG', 'GoalSecondHalf','FTR'
]

#
target_column = "SHHG"
drop_columns.remove(target_column)
X,Y = load_data(columns_to_drop=drop_columns, is_training=True, target_name=target_column)
Y=[Y]

KEEEP
LENGTH800


In [26]:
#X.head()

In [27]:
#Y.head()

# 2. Develop a Baseline Neural Network Model

In [28]:
# define base model
def baseline_model():
	# create model
	model = Sequential()
    #input_dim=431,
	model.add(Dense(13,  kernel_initializer='normal', activation='relu'))
	model.add(Dense(1, kernel_initializer='normal'))
	# Compile model
	model.compile(loss='mean_squared_error', optimizer='adam', verbos=1)
	return model

In [29]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
# evaluate model with standardized dataset
estimator = KerasRegressor(build_fn=baseline_model, nb_epoch=100, batch_size=1, verbose=1)

In [30]:
kfold = TimeSeriesSplit(n_splits=5)
results = cross_val_score(estimator, X, Y, cv=kfold, n_jobs=1)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

ValueError: Found input variables with inconsistent numbers of samples: [800, 1]

In [31]:
kfold = TimeSeriesSplit(n_splits=2)

# 3. Modeling The Standardized Dataset

In [21]:
# evaluate model with standardized dataset
np.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=baseline_model, epochs=5, batch_size=5, verbose=1)))
pipeline = Pipeline(estimators)

results = cross_val_score(pipeline, X, Y, cv=kfold, n_jobs=1)
print("Standardized: %.2f (%.2f) MSE" % (results.mean(), results.std()))
print(results)

c:\users\martin\anaconda3\envs\tensorflow\lib\site-packages\sklearn\model_selection\_validation.py:542: FutureWarning: From version 0.22, errors during fit will result in a cross validation score of NaN by default. Use error_score='raise' if you want an exception raised or error_score=np.nan to adopt the behavior from version 0.22.
  FutureWarning)


ValueError: Please provide as model targets either a single array or a list of arrays. You passed: y=[0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1]

# 4. Evaluate a Deeper Network Topology

In [22]:
# define the model
def larger_model():
	# create model
	model = Sequential()
    #input_dim=13,
	model.add(Dense(13,  kernel_initializer='normal', activation='relu'))
	model.add(Dense(6, kernel_initializer='normal', activation='relu'))
	model.add(Dense(1, kernel_initializer='normal'))
	# Compile model
	model.compile(loss='mean_squared_error', optimizer='adam')
	return model

In [ ]:
np.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=larger_model, epochs=10, batch_size=1, verbose=1)))
pipeline = Pipeline(estimators)
#kfold = KFold(n_splits=2, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold, n_jobs=1)

print("Larger: %.2f (%.2f) MSE" % (results.mean(), results.std()))

In [ ]:
results

In [3]:
model = larger_model()
model.fit(X, Y, nb_epoch=10, batch_size=1, validation_split=0.2, verbose=1)

NameError: name 'larger_model' is not defined

In [4]:
#pipeline.fit(X[:100])
model.predict([X[0:20]])

NameError: name 'model' is not defined

# 5. Evaluate a Wider Network Topology

In [101]:
# define wider model

def wider_model():
	# create model
	model = Sequential()
   #,input_dim=13
	model.add(Dense(20,  kernel_initializer='normal', activation='relu'))
	model.add(Dense(1, kernel_initializer='normal'))
	# Compile model
	model.compile(loss='mean_squared_error', optimizer='adam')
	return model

In [103]:
np.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=wider_model, epochs=100, batch_size=5, verbose=1)))
pipeline = Pipeline(estimators)
#kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold, n_jobs=1)
print("Wider: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Epoch 1/100
268/268 [==============================] - 2s 8ms/step - loss: 38.7355
Epoch 2/100
268/268 [==============================] - 0s 1ms/step - loss: 30.1291
Epoch 3/100
268/268 [==============================] - 0s 1ms/step - loss: 17.8789
Epoch 4/100
268/268 [==============================] - 0s 1ms/step - loss: 10.6424
Epoch 5/100
268/268 [==============================] - 0s 1ms/step - loss: 7.4601
Epoch 6/100
268/268 [==============================] - 0s 965us/step - loss: 5.5882
Epoch 7/100
268/268 [==============================] - 0s 1ms/step - loss: 4.4613
Epoch 8/100
268/268 [==============================] - 0s 1ms/step - loss: 3.5402
Epoch 9/100
268/268 [==============================] - 0s 1ms/step - loss: 2.8797
Epoch 10/100
268/268 [==============================] - 0s 995us/step - loss: 2.3600
Epoch 11/100
268/268 [==============================] - 0s 1ms/step - loss: 1.9540
Epoch 12/100
268/268 [==============================] - 0s 1ms/step - loss: 1.6388
Epoch

534/534 [==============================] - 1s 1ms/step - loss: 0.0613
Epoch 97/100
534/534 [==============================] - 1s 1ms/step - loss: 0.0660
Epoch 98/100
534/534 [==============================] - 1s 1ms/step - loss: 0.0607
Epoch 99/100
534/534 [==============================] - 1s 1ms/step - loss: 0.0463
Epoch 100/100
266/266 [==============================] - 1s 4ms/step
Wider: -11.74 (0.43) MSE


In [135]:
results

array([-1.55653163, -1.63254036, -2.13038876, -1.845678  , -1.54794455])